<a href="https://colab.research.google.com/github/lobasfs/-ML-/blob/main/Regression_IC50_LobasFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
#from sklearn.svm import SVR
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('Обработанный.csv')
data.head()

,"IC50, mM","CC50, mM",SI,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,...,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea
0,6.239374,175.482382,28.125000,5.094096,5.094096,0.387225,0.387225,0.417362,42.928571,384.652,...,0,0,0,0,0,0,0,0,3,0
1,0.771831,5.402819,7.000000,3.961417,3.961417,0.533868,0.533868,0.462473,45.214286,388.684,...,0,0,0,0,0,0,0,0,3,0
2,223.808778,161.142320,0.720000,2.627117,2.627117,0.543231,0.543231,0.260923,42.187500,446.808,...,2,0,0,0,0,0,0,0,3,0
3,1.705624,107.855654,63.235294,5.097360,5.097360,0.390603,0.390603,0.377846,41.862069,398.679,...,0,0,0,0,0,0,0,0,4,0
4,107.131532,139.270991,1.300000,5.150510,5.150510,0.270476,0.270476,0.429038,36.514286,466.713,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df = data.copy()

In [4]:
# Разделяем данные на признаки и целевой столбец
X = df.drop(['CC50, mM', 'SI'], axis=1)
y = np.log(df['IC50, mM'])
X

,"IC50, mM",MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,...,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea
0,6.239374,5.094096,5.094096,0.387225,0.387225,0.417362,42.928571,384.652,340.300,384.350449,...,0,0,0,0,0,0,0,0,3,0
1,0.771831,3.961417,3.961417,0.533868,0.533868,0.462473,45.214286,388.684,340.300,388.381750,...,0,0,0,0,0,0,0,0,3,0
2,223.808778,2.627117,2.627117,0.543231,0.543231,0.260923,42.187500,446.808,388.344,446.458903,...,2,0,0,0,0,0,0,0,3,0
3,1.705624,5.097360,5.097360,0.390603,0.390603,0.377846,41.862069,398.679,352.311,398.366099,...,0,0,0,0,0,0,0,0,4,0
4,107.131532,5.150510,5.150510,0.270476,0.270476,0.429038,36.514286,466.713,424.377,466.334799,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,31.000104,12.934891,12.934891,0.048029,-0.476142,0.382752,49.133333,414.542,380.270,414.240624,...,0,0,0,0,0,0,0,0,0,0
945,31.999934,13.635345,13.635345,0.030329,-0.699355,0.369425,44.542857,485.621,446.309,485.277738,...,0,0,0,0,0,0,0,0,0,0
946,30.999883,13.991690,13.991690,0.026535,-0.650790,0.284923,41.973684,545.742,502.398,545.281109,...,0,1,0,0,0,0,0,0,0,0
947,31.998959,13.830180,13.830180,0.146522,-1.408652,0.381559,39.000000,522.635,480.299,522.282883,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Проводим стандартизацию признаков
scales = StandardScaler()
X_scaled = scales.fit_transform(X)
X_scaled

array([[-0.54802931, -1.7555751 , -1.7555751 , ..., -0.27782192,
         2.2416977 , -0.08620324],
       [-0.56170396, -2.09860733, -2.09860733, ..., -0.27782192,
         2.2416977 , -0.08620324],
       [-0.00387549, -2.50270017, -2.50270017, ..., -0.27782192,
         2.2416977 , -0.08620324],
       ...,
       [-0.48610183,  0.939063  ,  0.939063  , ..., -0.27782192,
        -0.1711996 , -0.08620324],
       [-0.48360308,  0.89014984,  0.89014984, ..., -0.27782192,
        -0.1711996 , -0.08620324],
       [-0.31352967,  0.75407414,  0.75407414, ..., -0.27782192,
        -0.1711996 , -0.08620324]])

In [6]:
# pca = PCA(n_components=0.90)
# X_pca = pca.fit_transform(X_scaled)

# PCA не подходит, т.к. это линейный метод понижения размерности,
# а по эксперентам далее, я понял, что данные нелинейно заввисимы, поэтому это ухудшает качество модели

In [7]:
# Отбор 30 лучших признаков по mutual_info, для понижения размерности ускорения обучения
selector = SelectKBest(score_func=mutual_info_regression, k=30)
X_selected = selector.fit_transform(X_scaled, y)
selected_feature_names = X.columns[selector.get_support()]
selected_feature_names

Index(['IC50, mM', 'MaxAbsEStateIndex', 'MaxEStateIndex', 'MolWt',
       'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons',
       'MaxPartialCharge', 'MaxAbsPartialCharge', 'MinAbsPartialCharge',
       'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI',
       'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW', 'AvgIpc', 'BalabanJ',
       'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Kappa1', 'Kappa2', 'LabuteASA',
       'SlogP_VSA4', 'EState_VSA10', 'VSA_EState8', 'HeavyAtomCount'],
      dtype='object')

In [8]:
X_selected

array([[-0.54802931, -1.7555751 , -1.7555751 , ..., -0.96090201,
         1.9860852 ,  0.32963129],
       [-0.56170396, -2.09860733, -2.09860733, ..., -0.96090201,
         2.10459486,  0.32963129],
       [-0.00387549, -2.50270017, -2.50270017, ..., -0.96090201,
         2.21058662,  0.7660737 ],
       ...,
       [-0.48610183,  0.939063  ,  0.939063  , ...,  0.83519053,
         0.57555493,  1.42073732],
       [-0.48360308,  0.89014984,  0.89014984, ...,  1.28421366,
         0.35449323,  1.31162671],
       [-0.31352967,  0.75407414,  0.75407414, ...,  0.38616739,
         0.61426367,  0.6569631 ]])

In [9]:
# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

In [10]:
# Возьмем пять моделей, обычную линейную регрессию, регуляризацию Ridge и Lasso,
# RandomForest, XGBoost и их разные гиперпараметры для подбора
models = {
    "LinearRegression": (LinearRegression(), {}),
    "Ridge": (Ridge(), {"alpha": [0.01, 0.1, 1.0, 10.0, 100.0]}),
    "Lasso": (Lasso(max_iter=10000), {"alpha": [0.0001, 0.001, 0.01, 0.1, 1.0]}),
    "RandomForest": (
        RandomForestRegressor(random_state=42),
        {
            "n_estimators": [50, 100, 200],
            "max_depth": [5, 10, 15],
            "min_samples_split": [2, 5]
        }
    ),
    "XGBoost": (
        XGBRegressor(random_state=42, verbosity=0),
        {
            "n_estimators": [50, 100, 200],
            "max_depth": [3, 6, 10],
            "learning_rate": [0.01, 0.1, 0.3],
            "subsample": [0.8, 1.0]
        }
    )
}

In [11]:
# Обучение и сравнение моделей
results = []
for name, (model, params) in models.items():
    grid = GridSearchCV(model, params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)
    results.append({
        "Model": name,
        "Best Params": grid.best_params_,
        "MAE:": mean_absolute_error(y_test, y_pred),
        "MSE:": mean_squared_error(y_test, y_pred),
        "R2:": r2_score(y_test, y_pred)
    })

results_df = pd.DataFrame(results)
results_df

,Model,Best Params,MAE:,MSE:,R2:
0,LinearRegression,{},1.006660,2.150360,0.447608
1,Ridge,{'alpha': 100.0},1.053224,2.225804,0.428227
2,Lasso,{'alpha': 0.1},1.043713,2.340351,0.398802
3,RandomForest,"{'max_depth': 15, 'min_samples_split': 2, 'n_e...",0.025884,0.015279,0.996075
4,XGBoost,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",0.035833,0.018748,0.995184


In [12]:
rf = RandomForestRegressor(max_depth=10, min_samples_split = 5, n_estimators=200, random_state=42)

rf.fit(X_train, y_train)

y_rf_pred = rf.predict(X_test)

print('Mean Absolute Error:', mean_absolute_error(y_test, y_rf_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_rf_pred))
print('R2 score:', r2_score(y_test, y_rf_pred))

Mean Absolute Error: 0.025204374800329447
Mean Squared Error: 0.013488094747390666
R2 score: 0.9965351275763175


In [13]:
xgb =  XGBRegressor(learning_rate =  0.1, max_depth = 3, n_estimators=50, subsample=0.8, random_state=42, verbosity=0)

xgb.fit(X_train, y_train)

y_xgb_pred = xgb.predict(X_test)

print('Mean Absolute Error:', mean_absolute_error(y_test, y_xgb_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_xgb_pred))
print('R2 score:', r2_score(y_test, y_xgb_pred))

Mean Absolute Error: 0.036244079550628236
Mean Squared Error: 0.016524129223254778
R2 score: 0.9957552196404834
